In [ ]:
import pandas as pd
import numpy as np
from math import cos, sin, radians
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d

In [ ]:
df_pontos =  pd.read_csv('dados_combinados.csv')

In [ ]:
df_pontos.head()

,distance,intensity,angle,Row,Pitch,Yaw,Angulo Esperado
0,1.195,0.898039,0.000872,-51.212769,1.159058,-89.489136,-63.0
1,1.034,0.733333,0.001222,-51.212769,1.159058,-89.489136,-63.0
2,1.194,0.905882,0.003665,-51.212769,1.159058,-89.489136,-63.0
3,1.034,0.745098,0.005411,-51.212769,1.159058,-89.489136,-63.0
4,1.194,0.909804,0.006458,-51.212769,1.159058,-89.489136,-63.0


In [ ]:
df_pontos.tail()

,distance,intensity,angle,Row,Pitch,Yaw,Angulo Esperado
802374,1.172,0.886275,6.269222,56.508179,1.356812,-71.779175,54.0
802375,1.172,0.886275,6.272015,56.508179,1.356812,-71.779175,54.0
802376,1.171,0.882353,6.274808,56.508179,1.356812,-71.779175,54.0
802377,1.170,0.882353,6.277600,56.508179,1.356812,-71.779175,54.0
802378,1.170,0.886275,6.280393,56.508179,1.356812,-71.779175,54.0


802378 pontos

--------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_pontos.groupby('Angulo Esperado').count()

,distance,intensity,angle,Row,Pitch,Yaw
Angulo Esperado,,,,,,
-63.00,51843,51843,51843,51843,51843,51843
-56.70,8632,8632,8632,8632,8632,8632
-56.37,8671,8671,8671,8671,8671,8671
-55.59,8663,8663,8663,8663,8663,8663
-54.00,6493,6493,6493,6493,6493,6493
...,...,...,...,...,...,...
50.40,29959,29959,29959,29959,29959,29959
54.00,8690,8690,8690,8690,8690,8690
55.59,6531,6531,6531,6531,6531,6531


66 planos de corte registrados

--------------------------------------------------------------------------------------------------------------------------

In [ ]:
print(df_pontos['angle'].min(), df_pontos['angle'].max())

0.0001747074581246 6.28301114077353


angle é o ângulo de varredura do lidar, e varia de 0 a 2 pi

--------------------------------------------------------------------------------------------------------------------------

Row, Pitch e Yaw estão em graus, indo de 0 a 360

In [ ]:
print(df_pontos['Row'].min(), df_pontos['Row'].max())

-51.2127685546875 59.4305419921875


Row é nossa 'inclinação frontal', a que estamos realizando com o lidar

Note que apesar de ter colocado para variar de -63 a 63 graus, esses valores nunca foram realmente atingidos

--------------------------------------------------------------------------------------------------------------------------

In [ ]:
print(df_pontos['Pitch'].min(), df_pontos['Pitch'].max())

1.1590576171875 3.9276123046875


Pitch é a 'inclinação lateral'. As variações pequenas mostradas podem ter origem na vibração da base

--------------------------------------------------------------------------------------------------------------------------

In [ ]:
print(df_pontos['Yaw'].min(), df_pontos['Yaw'].max())

-90.692138671875 -69.818115234375


Yaw é a rotação da base. As variações demonstradas não fazem sentido no dado exemplo, já que nenhuma rotação foi feita e colocamos peso na base para segurar o movimento brusco dos servos

--------------------------------------------------------------------------------------------------------------------------

De certa maneira,vamos sempre respeitar Row e Pitch, mas testar respeitar o Yaw ou não.

Vale lembrar que o sensor está no plano do lidar, a 15.5 cm do eixo de inclinação.

--------------------------------------------------------------------------------------------------------------------------

Criando Nossa Nuvem de Pontos:

Iremos criar três colunas, uma X, uma Y e uma Z

Elas vão ser o resultado da transformação por matrizes partindo da distância, Row, Pitch e Yaw e vão representar a posição do ponto no espaço 3d.

****************************************************************************************************************************
Abaixo deixarei o exemplo de como estava a transformação anterior, a nossa tarefa é conseguir a transformação correta para os pontos. O ambiente escaneado é uma sala + cozinha (aberta).
****************************************************************************************************************************

In [ ]:
dist_sensor_eixo = 0.155 #lidar to tilt axis distance
z_ajuste = 1

def transform_row(row):

    angle_rad = row['angle']

    Row_rad   = radians(row['Row'])
    Pitch_rad = radians(row['Pitch'])
    Yaw_rad   = radians(row['Yaw'])

    Rx = np.array([[1, 0, 0],
                   [0, cos(Row_rad), -sin(Row_rad)],
                   [0, sin(Row_rad), cos(Row_rad)]])

    Ry = np.array([[cos(Pitch_rad), 0, sin(Pitch_rad)],
                   [0, 1, 0],
                   [-sin(Pitch_rad), 0, cos(Pitch_rad)]])

    Rz = np.array([[cos(Yaw_rad), sin(Yaw_rad), 0],
                   [-sin(Yaw_rad), cos(Yaw_rad), 0],
                   [0, 0, 1]])

    R_total = Rz.dot(Ry.dot(Rx))

    distance = row['distance']
    x_local = distance * sin(angle_rad)
    y_local = distance * cos(angle_rad)
    z_local = 0.0  # LiDAR 2D

    Pponto = np.array([[x_local],
                        [y_local],
                        [z_local]])

    PLidar = np.array([[0],
                       [0],
                       [dist_sensor_eixo]])

    Pfponto = R_total.dot(Pponto  + PLidar)

    Pfponto[2, 0] += z_ajuste

    return pd.Series({
        'x': Pfponto[0, 0],
        'y': Pfponto[1, 0],
        'z': Pfponto[2, 0]
    })

In [ ]:
transformed = df_pontos.apply(transform_row, axis=1)
df_transformed = pd.concat([df_pontos, transformed], axis=1)

In [ ]:
df_transformed.head()

,distance,intensity,angle,Row,Pitch,Yaw,Angulo Esperado,x,y,z
0,1.195,0.898039,0.000872,-51.212769,1.159058,-89.489136,-63.0,-0.110797,1.197738,2.072092
1,1.034,0.733333,0.001222,-51.212769,1.159058,-89.489136,-63.0,-0.112371,1.036775,2.075176
2,1.194,0.905882,0.003665,-51.212769,1.159058,-89.489136,-63.0,-0.112895,1.196697,2.069514
3,1.034,0.745098,0.005411,-51.212769,1.159058,-89.489136,-63.0,-0.115085,1.036716,2.071802
4,1.194,0.909804,0.006458,-51.212769,1.159058,-89.489136,-63.0,-0.114984,1.196646,2.066916


In [ ]:
def visualize_3d(df):
    df = df

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    scatter = ax.scatter(df['x'], df['y'], df['z'], c=df['intensity'], cmap='viridis', marker='o')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    fig.colorbar(scatter, ax=ax, label='Intensity')

    plt.title("Visualização 3D de df_transformed")
    plt.show()

if __name__ == '__main__':
    visualize_3d(df_transformed)

In [ ]:
def visualize_3d_open3d(df):
    points = df[['x', 'y', 'z']].to_numpy()
    intensities = df['intensity'].to_numpy()

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)

    norm_intensities = (intensities - intensities.min()) / (intensities.ptp() + 1e-8)
    colors = np.repeat(norm_intensities.reshape(-1, 1), 3, axis=1)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    o3d.visualization.draw_geometries([pcd],
                                      window_name="Visualização 3D com Open3D",
                                      width=800,
                                      height=600,
                                      left=50,
                                      top=50,
                                      point_show_normal=False)

if __name__ == "__main__":
    visualize_3d_open3d(df_transformed)